In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from rasterio.enums import Resampling
from PIL import ImageColor
from skimage.exposure import rescale_intensity
import rasterio as rio
import os
import pandas as pd
from PIL import ImageColor
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, Input, concatenate
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from skimage.exposure import rescale_intensity
from rasterio.enums import Resampling
import rasterio.warp as warp

import torch

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU


In [ ]:
# Location of data
lc_dir = r'C:\Users\Usuari\Documents\TFM_Codigos\MODELO\data/7Labels.json'
# Load Land Cover Parameter
lc = json.load(open(lc_dir))
lc_df = pd.DataFrame(lc)
lc_df["values_normalize"] = lc_df.index #+ 1
lc_df["palette"] = "#" + lc_df["palette"]

# Mapping from old to new values
values = lc_df["values"].to_list()
values_norm = lc_df["values_normalize"].to_list()
palette = lc_df["palette"].to_list()
labels = lc_df["label"].to_list()
dict_values = {}
dict_label = {}
dict_palette = {}
dict_palette_hex = {}
for x in range(0, len(values)):
    dict_values[values[x]] = values_norm[x]
    dict_label[values_norm[x]] = labels[x]
    dict_palette[values_norm[x]] = ImageColor.getrgb(palette[x])
    dict_palette_hex[values_norm[x]] = palette[x]

# Create colormap from values and palette
cmap = ListedColormap(palette)

# Patches legend
patches = [
    mpatches.Patch(color=palette[i], label=labels[i]) for i in range(len(values))
]
legend = {
    "handles": patches,
    "bbox_to_anchor": (1.05, 1),
    "loc": 2,
    "borderaxespad": 0.0,
}
lc_df

In [ ]:
import numpy as np
import os
import re

def load_and_combine_datasets(folder_path):
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Expresiones regulares para identificar los archivos
    image_train_pattern = re.compile(r"images_train_PNOA_(\d+)\.npy")
    image_test_pattern = re.compile(r"images_test_PNOA_(\d+)\.npy")
    lcs_train_pattern = re.compile(r"lcs_train_PNOA_(\d+)\.npy")
    lcs_test_pattern = re.compile(r"lcs_test_PNOA_(\d+)\.npy")
    
    # Diccionarios para almacenar los arrays cargados
    image_train_arrays = []
    image_test_arrays = []
    lcs_train_arrays = []
    lcs_test_arrays = []
    
    # Recorrer todos los archivos en la carpeta
    for file in files:
        # Cargar y clasificar los archivos según su tipo
        if image_train_pattern.match(file):
            image_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif image_test_pattern.match(file):
            image_test_arrays.append(np.load(os.path.join(folder_path, file)))
        elif lcs_train_pattern.match(file):
            lcs_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif lcs_test_pattern.match(file):
            lcs_test_arrays.append(np.load(os.path.join(folder_path, file)))
    
    # Combinar los arrays de cada categoría
    combined_images_train = np.concatenate(image_train_arrays, axis=0)
    combined_images_test = np.concatenate(image_test_arrays, axis=0)
    combined_lcs_train = np.concatenate(lcs_train_arrays, axis=0)
    combined_lcs_test = np.concatenate(lcs_test_arrays, axis=0)
    
    return combined_images_train, combined_images_test, combined_lcs_train, combined_lcs_test

# Uso de la función
folder_path = r"C:\Users\Usuari\Documents\TFM_Codigos\DATASET\Data-arrays"
# Cambia esto por la ruta a tu carpeta
images_train, images_test, lcs_train, lcs_test = load_and_combine_datasets(folder_path)

In [ ]:
print(f'Train_predictors_shape: {images_train.shape}\nTrain_label_shape: {lcs_train.shape}\nTest_predictors_shape: {images_test.shape}\nTest_label_shape: {lcs_test.shape}')

In [ ]:
# Make lcs data into categorical
# Keras model use different output data shape for category data
# Convert it using utility from keras to make into categorical shape
lcs_train_category = to_categorical(lcs_train)
lcs_test_category = to_categorical(lcs_test)
print(lcs_train_category.shape)

In [ ]:
# Definir el modelo DeepLab v3+ para imágenes de 128x128
def create_deeplabv3plus(input_shape, num_classes):
    # Usar ResNet50V2 como backbone
    base_model = tf.keras.applications.ResNet50V2(input_shape=input_shape, include_top=False, weights='imagenet')
    
    # Capas de la red base (ajustadas para 128x128)
    layer_names = [
        'conv2_block3_out',   # 1/4 de la resolución original (32x32)
        'conv3_block4_out',   # 1/8 de la resolución original (16x16)
        'conv4_block6_out',   # 1/16 de la resolución original (8x8)
    ]
    layer_outputs = [base_model.get_layer(name).output for name in layer_names]
    
    # Crear el modelo de la red base
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layer_outputs)
    
    # Congelar la red base
    down_stack.trainable = False
    
    # Otra convolución delpues del encoder  a tamaño 4x4x256??? evitar sobreajuste con dropout
    x = layers.Conv2D(256, (1, 1), padding='same', activation='relu')(down_stack.output[-2])  # 8×8 correcto
    x = layers.Dropout(0.5)(x)

    # Decodificador corregido
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)  # 8×8 → 16×16
    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 16×16 → 32×32
    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 32×32 → 64×64
    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 64×64 → 128×128

    # Salida final
    x = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)  # (128×128×7)

    # Crear el modelo final
    model = tf.keras.Model(inputs=down_stack.input, outputs=x)
    for i, layer_output in enumerate(down_stack.outputs):
        print(f"Salida {i}: {layer_output.shape}")
    
    return model

# Parámetros del modelo
input_shape = (128, 128, 3)  # Tamaño de las imágenes
num_classes = 7  # Número de clases

# Crear el modelo
model = create_deeplabv3plus(input_shape, num_classes)

# Verificar la forma de salida del modelo
dummy_input = tf.random.normal([1, 128, 128, 3])  # Entrada dummy de forma (1, 128, 128, 3)
output = model(dummy_input)
print("Forma de salida del modelo:", output.shape)  # Debería ser (1, 128, 128, 7)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.metrics import MeanIoU, CategoricalAccuracy, Precision, Recall, AUC
# Parámetros del modelo
input_shape = (128, 128, 3)  # Tamaño de las imágenes
num_classes = 7  # Número de clases

# Crear el modelo
model = create_deeplabv3plus(input_shape, num_classes)

# Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=CategoricalCrossentropy(),
    metrics=[MeanIoU(num_classes=num_classes), CategoricalAccuracy(), Precision(), Recall(), AUC()]
)

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.1, patience=5)
]

# Entrenar el modelo
history = model.fit(
    x=images_train,
    y=lcs_train_category,
    batch_size=16,  # Aumentar el batch size
    epochs=35,
    validation_data=(images_test, lcs_test_category),
    callbacks=callbacks,
    shuffle=True,
    validation_split=0.1,
    verbose=1
)

In [ ]:
model.save('FCN_rESnEt_v0.keras')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Número de imágenes a mostrar
num_samples = 10
indices = np.random.choice(len(images_test), num_samples, replace=False)

plt.figure(figsize=(12, num_samples * 3))

for i, idx in enumerate(indices):
    img = images_test[idx]
    true_mask = lcs_test[idx]  # Máscara real

    # Predecir la máscara
    pred_mask = model.predict(img[np.newaxis, ...])  # Agregar dimensión batch
    pred_mask = np.argmax(pred_mask, axis=-1)[0]  # Convertir softmax a etiquetas de clase

    # Mostrar la imagen original
    plt.subplot(num_samples, 3, i * 3 + 1)
    plt.imshow(img)
    plt.title("Imagen original")
    plt.axis("off")

    # Mostrar la máscara real con los colores correctos
    plt.subplot(num_samples, 3, i * 3 + 2)
    plt.imshow(true_mask, cmap=cmap, vmin=0, vmax=len(palette) - 1)  # Usar el colormap definido
    plt.title("Máscara real")
    plt.axis("off")

    # Mostrar la predicción con los colores correctos
    plt.subplot(num_samples, 3, i * 3 + 3)
    plt.imshow(pred_mask, cmap=cmap, vmin=0, vmax=len(palette) - 1)  # Usar el colormap definido
    plt.title("Predicción")
    plt.axis("off")

plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(2, 5))
ax.legend(**legend)
ax.axis("off")
plt.show()


In [ ]:
# Obtener los valores del historial con los nombres correctos
loss = history.history['loss']
val_loss = history.history['val_loss']
iou = history.history['mean_io_u_1']  # Nombre correcto
val_iou = history.history['val_mean_io_u_1']  # Nombre correcto

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(12, 5))

# 📉 Gráfico de Pérdida
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo-', label='Entrenamiento')
plt.plot(epochs, val_loss, 'ro-', label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# 📈 Gráfico de MeanIoU
plt.subplot(1, 2, 2)
plt.plot(epochs, iou, 'bo-', label='Entrenamiento')
plt.plot(epochs, val_iou, 'ro-', label='Validación')
plt.title('Mean IoU durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Mean IoU')
plt.legend()

plt.show()

plt.show()


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Obtener predicciones del modelo en el conjunto de prueba
y_pred = model.predict(images_test)  # (N, 128, 128, num_classes)
y_pred = np.argmax(y_pred, axis=-1)  # Convertir a etiquetas de clase (N, 128, 128)

# Obtener las máscaras verdaderas (ground truth)
y_true = np.argmax(lcs_test_category, axis=-1)  # Convertir one-hot a etiquetas (N, 128, 128)

# Aplanar las máscaras para usar con sklearn (pixel a pixel)
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

# Calcular precisión, recall y F1-score por clase
precision = precision_score(y_true_flat, y_pred_flat, average=None)  # Por clase
recall = recall_score(y_true_flat, y_pred_flat, average=None)  # Por clase
f1 = f1_score(y_true_flat, y_pred_flat, average=None)  # Por clase

# Mostrar los valores de F1-score por clase
for i, f1_value in enumerate(f1):
    print(f"Clase {i}: F1-score = {f1_value:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Obtener las predicciones del modelo
y_pred = model.predict(images_test)  # (N, 128, 128, num_classes)
y_pred = np.argmax(y_pred, axis=-1)  # Convertir a etiquetas de clase (N, 128, 128)

# Obtener las máscaras verdaderas (ground truth)
y_true = np.argmax(lcs_test_category, axis=-1)  # Convertir one-hot a etiquetas (N, 128, 128)

# Aplanar las predicciones y las etiquetas verdaderas para la matriz de confusión
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

# Calcular la matriz de confusión
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=np.arange(num_classes))

# Crear un gráfico con la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()


In [ ]:
# Normalizar la matriz de confusión (entre 0 y 1)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Visualización de la matriz normalizada
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels, cbar_kws={'label': 'Frecuencia Relativa'})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
import pandas as pd
history_dict = history.history
# Extraer las métricas
epochs = range(1, len(history_dict['loss']) + 1)
loss = history_dict['loss']
val_loss = history_dict['val_loss']
iou = history_dict.get('mean_io_u', None)  # Si tienes 'mean_io_u_1' como métrica
val_iou = history_dict.get('val_mean_io_u', None)  # Similar para 'val_mean_io_u_1'

# Crear un DataFrame
data = {
    'Epoch': epochs,
    'Loss': loss,
    'Validation Loss': val_loss,
    'Mean IoU': iou,
    'Validation Mean IoU': val_iou
}

# Convertir en un DataFrame de Pandas
df = pd.DataFrame(data)
df.to_csv('training_metrics.csv', index=False)
# Mostrar el DataFrame
print(df)

In [ ]:
# Crear un DataFrame con los resultados de F1 Score, Precision y Recall por clase
f1_scores = pd.DataFrame({
    'Class': labels,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
})

# Guardar los F1 Scores en un archivo CSV
f1_scores.to_csv('f1_scores.csv', index=False)

# Mostrar el DataFrame con los resultados de F1 Score
print(f1_scores)

In [ ]:
# Show history
history = pd.DataFrame(history.history)
print(history)

plt.figure(figsize = (10, 8))
plt.plot(range(len(history['categorical_accuracy'].values.tolist())), history['categorical_accuracy'].values.tolist(), label = 'Train_Accuracy')
plt.plot(range(len(history['loss'].values.tolist())), history['loss'].values.tolist(), label = 'Train_Loss')
plt.plot(range(len(history['val_categorical_accuracy'].values.tolist())), history['val_categorical_accuracy'].values.tolist(), label = 'Test_Accuracy')
plt.plot(range(len(history['val_loss'].values.tolist())), history['val_loss'].values.tolist(), label = 'Test_Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Predict test data
prediction = np.argmax(model.predict(images_test), 3).flatten()
label = lcs_test.flatten()
# Confusion matrix
cm = confusion_matrix(label, prediction, normalize='true')
fig, ax = plt.subplots(figsize=(15, 10))
cm = ConfusionMatrixDisplay(cm)
cm.plot(ax = ax)

# Classification report
print(classification_report(label, prediction))